# API Face Samples

## Objective

To develop a Python-based application that detects and verifies faces within an image using Azure Face SDK and REST API. The application will cover three scenarios:

- **Source Image + Search Image**: It will detect the largest face in a source image and compare it with all faces in a search image to find similar faces.

- **Source Face ID + Search Image**: It will use a given face ID to compare it with all faces in a search image to find similar faces.

- **Source Person ID with Person Group ID + Search Image**: It will use a given person ID within a person group to verify against all faces in a search image.

The application will analyze local images, identify faces, and provide verification results with confidence scores and bounding box coordinates for the detected faces. The results will include a decision based on a specified confidence threshold.


In [ ]:
import os, requests
from azure.core.credentials import AzureKeyCredential
from azure.ai.vision.face import FaceClient
from azure.ai.vision.face.models import FaceDetectionModel, FaceRecognitionModel, FindSimilarMatchMode

def find_faces_in_image(
    source_image_path=None, 
    source_face_id=None, 
    source_person_id=None, 
    person_group_id=None, 
    search_image_path=None, 
    subscription_key=None, 
    endpoint=None, 
    threshold=0.5
):
    with FaceClient(endpoint, AzureKeyCredential(subscription_key), headers = {"X-MS-AZSDK-Telemetry": "sample=find-faces-in-image"}) as face_client:
        if source_image_path:
            with open(source_image_path, 'rb') as source_image:
                source_faces = face_client.detect(
                    image_content=source_image.read(),
                    detection_model=FaceDetectionModel.DETECTION_03,
                    recognition_model=FaceRecognitionModel.RECOGNITION_04,
                    return_face_id=True,
                )
                if source_faces and len(source_faces) > 0:
                    source_face_id = source_faces[0].face_id
                else:
                    raise ValueError("No face detected in the source image")

        if search_image_path:
            with open(search_image_path, 'rb') as search_image:
                search_faces = face_client.detect(
                    image_content=search_image.read(),
                    detection_model=FaceDetectionModel.DETECTION_03,
                    recognition_model=FaceRecognitionModel.RECOGNITION_04,
                    return_face_id=True,
                )
                if search_faces and len(search_faces) > 0:
                    search_face_ids = [face.face_id for face in search_faces]
                else:
                    raise ValueError("No face detected in the search image")

        similar_faces = []
        if source_face_id:
            similar_faces = face_client.find_similar(
                face_id=source_face_id,
                face_ids=search_face_ids,
                mode=FindSimilarMatchMode.MATCH_FACE,
            )

    if source_person_id and person_group_id:
        # Use REST API for person ID and person group ID verification
        headers = {
            'Ocp-Apim-Subscription-Key': subscription_key,
            'Content-Type': 'application/json',
            'X-MS-AZSDK-Telemetry': 'sample=find-faces-in-image'
        }
        for search_face_id in search_face_ids:
            body = {
                'personGroupId': person_group_id,
                'personId': source_person_id,
                'faceId': search_face_id,
            }
            url = endpoint + '/face/v1.0/verify'
            response = requests.post(url, headers=headers, json=body)
            response.raise_for_status()
            result = response.json()
            result['faceId'] = search_face_id
            similar_faces.append(result)

    # Collect results with bounding boxes and make decisions based on the confidence threshold
    results = []
    for face in similar_faces:
        face_id = face['faceId']
        confidence = face.get('confidence', 0)
        bbox = next(item['faceRectangle'] for item in search_faces if item['faceId'] == face_id)
        if confidence >= threshold:
            results.append({
                'confidence': confidence,
                'bbox': bbox,
            })

    return results

# Example usage
FACE_KEY = os.environ["FACE_API_KEY"]
FACE_ENDPOINT = os.environ["FACE_ENDPOINT_URL"]
source_image_path = 'path_to_source_image.jpg'
search_image_path = 'path_to_search_image.jpg'
threshold = 0.5

# Case 1: Source image + search image
results_case1 = find_faces_in_image(
    source_image_path=source_image_path, 
    search_image_path=search_image_path, 
    subscription_key=subscription_key, 
    endpoint=endpoint, 
    threshold=threshold
)

for result in results_case1:
    print(f"Case 1 - Confidence: {result['confidence']}, BBox: {result['bbox']}")

# Case 2: Source face ID + search image
source_face_id = 'your_source_face_id'
results_case2 = find_faces_in_image(
    source_face_id=source_face_id, 
    search_image_path=search_image_path, 
    subscription_key=subscription_key, 
    endpoint=endpoint, 
    threshold=threshold
)

for result in results_case2:
    print(f"Case 2 - Confidence: {result['confidence']}, BBox: {result['bbox']}")

# Case 3: Source person ID with person group ID + search image
source_person_id = 'your_source_person_id'
person_group_id = 'your_source_person_group_id'
results_case3 = find_faces_in_image(
    source_person_id=source_person_id, 
    person_group_id=person_group_id, 
    search_image_path=search_image_path, 
    subscription_key=subscription_key, 
    endpoint=endpoint, 
    threshold=threshold
)

for result in results_case3:
    print(f"Case 3 - Confidence: {result['confidence']}, BBox: {result['bbox']}")